In [7]:
!pip install --upgrade --force-reinstall numpy



Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5


In [10]:
!pip install river




Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ----------------------------- ---------- 1.6/2.2 MB 9.3 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.0 MB/s eta 0:00:00


In [12]:
############### EMRIL V 1.3 ####################
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from river.tree import HoeffdingTreeClassifier

from time import perf_counter
from sklearn import clone
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import sys
import scipy.stats as ss
from scipy.stats import ks_2samp
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import statistics
from decimal import Decimal
import scipy.cluster.hierarchy as sch
from sklearn import preprocessing
from numpy import sqrt
import matplotlib.ticker as mticker
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import random
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
import math 
from numpy.linalg import norm
import statistics
import random
import os
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial  import distance_matrix
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
import numpy as np
from timeit import default_timer as timer
from river import tree


# Assuming you want to use River's HoeffdingTreeClassifier
clf = tree.HoeffdingTreeClassifier()

# Example of usage with stream-based data
clfs = [clf]
clfs_label = ["Hoeffding Tree Classifier"]



pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.8f}'.format)
#np.set_printoptions(suppress=True)
#np.set_printoptions(threshold=1000000000)
warnings.filterwarnings('ignore')


def __get_coefficients(y_true, y_pred_a, y_pred_b):
    a, b, c, d = 0, 0, 0, 0
    for i in range(y_true.shape[0]):
        if y_pred_a[i] == y_true[i] and y_pred_b[i] == y_true[i]:
            a = a + 1
        elif y_pred_a[i] != y_true[i] and y_pred_b[i] == y_true[i]:
            b = b + 1
        elif y_pred_a[i] == y_true[i] and y_pred_b[i] != y_true[i]:
            c = c + 1
        else:
            d = d + 1
 
    return a, b, c, d

def q_statistics(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    q = float(a * d - b * c) / (a * d + b * c)
    return q

def double_fault_measure(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    df = float(d) / (a + b + c + d)
    return df

def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += str(ele)+ "-" 
    
    # return string  
    return str1 
# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result
def pltcolor(lst):
    cols=[]
    for l in lst:
        if l==0:
            cols.append('blue')
        elif l==1:
            cols.append('red')
    return cols


def detect_overlap(data_min,data_maj,n_features):
    fishers = []
    found = 0
    max_f1 = 0
    max_feature_index = 0
    Xy_up = data_min
    Xy_down = data_maj
    frames = [Xy_up,Xy_down]
    Xy_d = pd.concat(frames)
    f_data_up_means = Xy_up.mean()
    f_data_down_means= Xy_down.mean()
    f_data_up_vars = Xy_up.var()
    f_data_down_vars =Xy_down.var()
    for i in range(n_features):
        mean_up = f_data_up_means[i]
        mean_down = f_data_down_means[i]
        var_up = f_data_up_vars[i]
        var_down = f_data_down_vars[i]
        fisher = ((mean_up-mean_down)**2) / (var_up + var_down)
        fishers.append([i,round(fisher,5),round(mean_up,5),round(mean_down,5)])
        if(fisher>max_f1) :
            max_f1 = fisher
            max_feature_index = i

    max_index= max_feature_index + 1
    #print("Actual Max:",max_f1)
    max_f1 = 1/(1+max_f1)   # push between (0,1]
    #print("Transformed Max:",max_f1)       
    fishers=sorted(fishers,key=lambda x: x[1], reverse=False)
    return round(max_f1,5), fishers

def update_minority_buffer(X,y,n_features):
    global minority_buffer_x
    global minority_buffer_y 
    global w
    global df_buffer_x
    global min_life_max
    
    decrease_life()   
    remove_zero_life_rows()
    for n in range (0, len(X)):
        df_buffer_x = df_buffer_x.append(X.iloc[n])
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("life")] = min_life_max
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("recall_weight")] = 1
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("total_weight")] = 1
    recalculate_total_weight()

def decrease_life():
    global df_buffer_x
    global min_life_decay_factor    
    df_buffer_x['life'] -= min_life_decay_factor
   # print(df_buffer_x)

def update_recall_weight(weight):
    global df_buffer_x
    global min_life_max
    df_buffer_x.loc[df_buffer_x.life == min_life_max, 'recall_weight'] = weight   
    #print(df_buffer_x)
   # sys.exit()
    
def recalculate_total_weight():
    global df_buffer_x
    global min_life_max
    df_buffer_x['total_weight'] = 0.5* df_buffer_x['life'] + 0.5* df_buffer_x['recall_weight']   
    #print(df_buffer_x)
    
#performs random oversampling  
def resample_from_buffer(X,y,samples_to_add):
   # print(X)
    global df_buffer_x
    temp_X = X
    temp_y = y
    temp_y = temp_y.iloc[0:0]
    temp_X = temp_X.iloc[0:0]
    try:
        to_add_for_one_element = round(samples_to_add/len(df_buffer_x),0)
        #print("to_add_for_one_element:",to_add_for_one_element)
        min_buf_copy = df_buffer_x
        min_buf_copy = min_buf_copy.drop(['life', 'recall_weight','total_weight'], axis=1)
        #print(len(min_buf_copy))
        for v in range (0,len(min_buf_copy)):
            #print(X.iloc[[v]])
            for c in range(0,int(to_add_for_one_element)):
                #print("adding resample")
                temp_X = temp_X.append(min_buf_copy.iloc[[v]])
                temp_y = temp_y.append({'class': 1}, ignore_index=True)
                #print(len(temp_X))
        #print(temp_X)
        #print("resample from buffer--return rows")
        #print(temp_X)
        #print(temp_y)
    except:
        a = 1 #do nothing
    
   # print(len(temp_X))
    #print(len(temp_y))
    return temp_X,temp_y

def remove_zero_life_rows():
    global df_buffer_x
    #print("before:",len(df_buffer_x))  
    df_buffer_x = df_buffer_x[df_buffer_x['total_weight'] >= 0.50]
    #print("after:",len(df_buffer_x))


#performs random oversampling  
def get_weighted_buffer_data(X,y):
    global df_buffer_x
    temp_X = X
    temp_y = y
    temp_y = temp_y.iloc[0:0]
    temp_X = temp_X.iloc[0:0]
    min_buf_copy = df_buffer_x #.loc[df_buffer_x['total_weight'] >= 0.50] 
    min_buf_copy = min_buf_copy.drop(['life', 'recall_weight','total_weight'], axis=1)    
    for v in range (0,len(min_buf_copy)):
            temp_X = temp_X.append(min_buf_copy.iloc[[v]])
            temp_y = temp_y.append({'class': 1}, ignore_index=True)    
    return temp_X,temp_y

#code for fixing clusters in the data
#user class_to_process=1 for minority space -- creates equal size clusters.
#for majority, set class_to_process=0, performs sampling within clusters without making them equal size.
def fix_disjuncts(X,y,samples_to_add,class_to_process,DontAddToBuffer,n_features):
    global disjuncts
    global output_folder
    global disjuncts_threshold
    global arr_ord_cols
    global minority_buffer_x
    global minority_buffer_y
    global w
    global df_buffer_x
    global disable_buffer
    #if minority is less than 5% in the batch 
    #then use minority buffer to do get data.    
    min_min_count = int(w*0.05) #we need atleast 5% samples for clustering
    
    #If minority, see if requires samples from buffer
    if(disable_buffer == False):
        if(class_to_process == 1):
            if(DontAddToBuffer == False):
                update_minority_buffer(X,y,n_features)
                #print("buffer updated !")

            if(len(X) < min_min_count):
                #print("**********************using buffer or resample from buffer*********************")
                if(len(df_buffer_x) < min_min_count):
                    return resample_from_buffer(X,y,samples_to_add)
                else:
                    X,y = get_weighted_buffer_data(X,y)



    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    if(len(X) < 10):
        return X,y
    
    data_scaled = X
    n_samples, n_features = X.shape
    try:
        data_scaled = preprocessing.scale(X)
    except:
        return X,y
    linked = linkage(data_scaled, 'average')
    last = linked[-10:, 2]
    last_rev = last[::-1]
    idxs = np.arange(1, len(last) + 1)
    accele = np.diff(last, 2)  # 2nd derivative of the distances
    accele_rev = accele[::-1]
    k = accele_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters

    model = AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto', connectivity=None, linkage='average', memory=None, n_clusters=k)
    y_hc=model.fit_predict(X)
    aggloclust=model.fit(X)
    (unique, counts) = np.unique(aggloclust.labels_, return_counts=True)
    frequencies = np.asarray((unique, counts)).T
    frequencies = frequencies[np.argsort(frequencies[:, 1])[::-1]]
    arr_cumsum = np.cumsum(frequencies,axis=0)
    cumsum = arr_cumsum[:,1]
    arr_sum_cumsum = np.sum(frequencies,axis=0)
    arr_clusters = []
    sum_cumsum = arr_sum_cumsum[1]
    for i in range(len(arr_cumsum)):
        val = cumsum[i]
        arr_clusters.append(frequencies[i,0])

    
    arr_y_frames  = []
    arr_x_frames  = []
    cluster1_count =0
    df_x1 = []
    df_y1 = []
    
    disjuncts = len(arr_clusters)
    if(disjuncts<disjuncts_threshold):
        return X,y
    
    valid_clusters =0
    for i in range(len(arr_clusters)):
        arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
        df_y = y.loc[y.index[arr_indexes]]
        if(len(df_y) > 2):
            valid_clusters += 1
    if(valid_clusters == 0):
        return X,y
    
    each_cluster_addition = int(np.round(samples_to_add / valid_clusters ))  
    for i in range(len(arr_clusters)):
            arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
            df_x = X.loc[X.index[arr_indexes]]
            df_y = y.loc[y.index[arr_indexes]]
            
            
            if(len(df_x) <3):
                continue
            
            to_add = 0
            if(i==0):
                cluster1_count = len(df_x)
                to_add = each_cluster_addition
            else:
                to_add = each_cluster_addition
                
                #code to make equal size clusters in minority
                if(class_to_process == 1):
                    current_cluster_count = len(df_x)
                    to_add += (cluster1_count-current_cluster_count)
                   
            df_cov = df_x.cov()
            df_mean = df_x.mean(axis=0)
            try:
                new_samples = np.random.multivariate_normal(df_mean, df_cov,to_add)
            except:
                return X,y
            
            cols = []
            for f in range(1,n_features+1):
                cols.append(str(f))
            df_new_samples = pd.DataFrame(new_samples,columns=cols)
            df_x = pd.concat([df_x.reset_index(drop=True),df_new_samples.reset_index(drop=True)],axis=0)
            df_x = df_x.reset_index(drop=True)
            if(len(arr_ord_cols)>0):
                for h in range (0,len(arr_ord_cols)):
                    ix = arr_ord_cols[h]
                    df_x[str(ix)] = round(df_x[str(ix)]) 
            for j in range(0,to_add):
                df_y = df_y.append(pd.Series([class_to_process],index=['class']),ignore_index = True) 
            
            df_y = df_y.reset_index(drop=True)            
            arr_x_frames.append(df_x)
            arr_y_frames.append(df_y)        
    
    np.set_printoptions(threshold=np.inf)            
    modified_X = pd.concat(arr_x_frames)
    modified_Y = pd.concat(arr_y_frames)
    
    modified_X = modified_X.reset_index(drop=True)
    modified_Y = modified_Y.reset_index(drop=True)
    #print("**********</Fix Disjuncts End>************")
    #print(modified_X.shape)
    return modified_X,modified_Y    


def icrc(X_maj,X_min,y_maj,y_min,n_features,overlap_val,fullFit,ups,downs):
    global f1_overlap_value
    global overlap_threshold_f1
    global imbalance_ratio_threshold
    ##################### Update Imbalance status of current window
    is_imbalance = False
    ir = 0
    if(ups ==0 or downs ==0):
        ir = imbalance_ratio_threshold + 1
    elif(ups > downs):
        ir = ups/downs
    elif(ups < downs):
        ir = downs/ups
    elif(ups == downs):
        ir = 1

    if(ir>=imbalance_ratio_threshold):
        is_imbalance = True
    ########################################

    #is_imbalance = False
    if(is_imbalance == False):
        frames = [X_min,X_maj]
        modified_X = pd.concat(frames)
        frames = [y_min,y_maj]
        modified_Y = pd.concat(frames).replace('UP',1).replace('DOWN',0) 
        return modified_X.to_numpy(),modified_Y.to_numpy().reshape(-1)

    if(is_imbalance==True):
        is_overlap = False
        fitted_already = False
        modified_Xmin,modified_Ymin = fix_disjuncts(X_min,y_min,(len(X_maj)-len(X_min)),1,fullFit,n_features)
        modified_Xmaj= X_maj
        modified_Ymaj = y_maj
        f1_overlap_value = overlap_val
        if (f1_overlap_value > overlap_threshold_f1): 
            is_overlap =  True
        if is_overlap:
            modified_Xmaj = modified_Xmaj.reset_index(drop=True)
            modified_Xmin = modified_Xmin.reset_index(drop=True)
            idx_maj = len(modified_Xmaj)
            
            all_data = pd.concat([modified_Xmaj,modified_Xmin])
            a = pdist(all_data,metric='euclidean')
            t_l = len(all_data)
            d_1 = linkage(a)
            
            d_2 = pd.DataFrame(d_1,columns=['i1','i2','i3','i4'])
            filtered_values = np.where( ((d_2["i1"] < idx_maj) & (d_2["i2"] > idx_maj) & (d_2["i2"] <t_l)  ) | ((d_2["i1"] > idx_maj) & (d_2["i1"] < t_l ) & (d_2["i2"] < idx_maj) ) )
            filtered =  d_2.loc[filtered_values]
            arr_indexes = filtered.iloc[:,0].to_numpy() 
            to_remove = int(len(arr_indexes)*disjunct_threshold_perc_maj)            
            drop_these = (arr_indexes[0:to_remove])
            modified_Xmaj.drop(index=drop_these,inplace=True)
            modified_Ymaj = modified_Ymaj.iloc[to_remove:]
        if(len(modified_Xmin) > len(modified_Xmaj)):
            modified_Xmaj,modified_Ymaj = fix_disjuncts(modified_Xmaj,modified_Ymaj,(len(modified_Xmin)-len(modified_Xmaj)),0,fullFit,n_features)

        frames = [modified_Xmin,modified_Xmaj]
        modified_X = pd.concat(frames)
        frames = [modified_Ymin,modified_Ymaj]
        modified_Y = pd.concat(frames).replace('UP',1).replace('DOWN',0) 
        
    return modified_X.to_numpy(),modified_Y.to_numpy().reshape(-1)



class DetectorClassifier:
    def __init__(self, clf):
        self.clf = clf

    def fit(self, X, y):
        for xi, yi in zip(X, y):
            if not isinstance(self.clf, HoeffdingTreeClassifier):
              raise TypeError("Classifier is not a HoeffdingTreeClassifier from river")

              self.clf = self.clf.learn_one(dict(enumerate(xi)), yi)
        return self

    def partial_fit(self, X, y):
        # Incremental learning: this will also use learn_one method
        for xi, yi in zip(X, y):
            self.clf.learn_one(dict(enumerate(xi)), yi)
        return self

    def predict(self, X):
        preds = []
        for xi in X:
            xi_dict = dict(enumerate(xi))
            pred = self.clf.predict_one(xi_dict)
            preds.append(pred if pred is not None else 0)  # fallback for None
        return np.asarray(preds)

    def predict_proba(self, X):
        probas = []
        for xi in X:
            xi_dict = dict(enumerate(xi))
            proba_dict = self.clf.predict_proba_one(xi_dict)
            # Ensure class order [0, 1]
            probas.append([proba_dict.get(0, 0.0), proba_dict.get(1, 0.0)])
        return np.asarray(probas)

    def score(self, X, y):
        preds = self.predict(X)
        return np.mean(preds == y)


  
def prequential(window_size,Xy,X, y, clf,n_features,overlap_threshold,imbalance_ratio_threshold,disjunct_threshold_perc_maj,file_out_prefix, n_train=1):
    
    """Prequential Evaluation: instances are first used to test, and then to train
    :return the label predictions for each test instance, and the associated running time
    """
    first_weights_flag = False
    
    arr_clf = []
    arr_clf_temp_to_use = []
    arr_weights = []
    arr_recall0_cw = []
    arr_recall1_cw = []
    
    a= 0.00001
    start = timer()

    global arr_elapsedtime
    global restart_threshold
    global arr_recall_0
    global arr_recall_1
    global output_folder
    global maj_weight
    global min_weight
    global ens_weight
    global ensemble_pool_size    
    global pruned_ensemble_size
    global rl_pruned_ids
    global preq_Qtable

    preq_Qtable = initialize_q_table(ensemble_pool_size, ensemble_pool_size)
    arr_actual_win = []
    arr_pred_win = []
    arr_roc = []
    arr_divs = []
    ensemble_tp = 0
    ensemble_fp =0
    ensemble_fn = 0
    ensemble_tn = 0
    
    
    ensemble_correct =0
    zeroClassCounter = 0
    oneClassCounter = 0
    current_correct_0 = 0
    current_correct_1 = 0
    
    current_correct_ensemble_0 = 0
    current_correct_ensemble_1 = 0
    
    predictions = ""    
    row_num = y.shape[0]   
   
    # Split an init batch
    X_init = X[0:n_train]
    y_init = y[0:n_train]
    
    # Used for training and evaluation
    X_train = X
    y_train = y
    
    Xy_train = Xy
    
        
    ensemble_preds = np.zeros(row_num)
    current_correct_0_cw = np.zeros(ensemble_pool_size)
    current_correct_1_cw = np.zeros(ensemble_pool_size)
    recall_0_cw = np.zeros(ensemble_pool_size)
    recall_1_cw = np.zeros(ensemble_pool_size)
    
   
    for g in range (0,ensemble_pool_size):
        #Create a classifier and add it to the pool
        obj_clf = DetectorClassifier(HoeffdingTreeClassifier())
        obj_clf = obj_clf.fit(X_init, y_init)
        arr_clf.append(obj_clf)
        arr_weights.append(1)
    
    
    current_w = 0
    window_counter = 0
    strProcessing = ""
    correct_predictions = 0

    strout = ""
    strgmean = ""

    oneClassCounter_CWindow = 0
    zeroClassCounter_CWindow = 0
    arr_cw_ens_preds = []
    current_correct_0_cw_est = 0
    current_correct_1_cw_est = 0
    window_recall_0 = 0
    window_recall_1 = 0
        
    for i in range(0, row_num):

        current_w += 1
        start_time = perf_counter()
        arr_preds = []
        arr_temp_preds = []
        
        for n in range(0,len(arr_clf)):
            clf = arr_clf[n]
            prediction_clf = clf.predict(X_train[i, :].reshape(1, -1))
            arr_preds.append(prediction_clf[0])
            arr_temp_preds.append(prediction_clf[0])
        arr_cw_ens_preds.append(arr_temp_preds)    
        #sys.exit()
       
        strout += "------------\n"
        estimators_arr = []
        for n in range(0,len(arr_clf)):
            estimators_arr.append(("HT-" + str(n), arr_clf[n]))
        
       
        df_estimators_arr = pd.DataFrame(arr_clf)
        df_weights_arr = pd.DataFrame(arr_weights)        
        df_weights_arr = df_weights_arr.sort_values(ascending=False,by=df_weights_arr.columns[0])        
        arr_clf_temp_to_use = []
        estimators_arr_temp = []
        arr_weights_temp = []

        for h in range(0,len(rl_pruned_ids)) :
            try:
                indexx = rl_pruned_ids[h].astype(int)
            except:
                indexx = rl_pruned_ids[h]

            index_original = df_weights_arr.iloc[[h]].index.values[0]
            arr_clf_temp_to_use.append(arr_clf[indexx])
            estimators_arr_temp.append(("HT-" + str(index_original), arr_clf[indexx]))
            arr_weights_temp.append(arr_weights[indexx])
        
        if(len(arr_clf_temp_to_use) == 0):
            for h in range(0,len(arr_clf)) :               
                index_original = df_weights_arr.iloc[[h]].index.values[0]
                arr_clf_temp_to_use.append(arr_clf[index_original])
                estimators_arr_temp.append(("HT-" + str(index_original), arr_clf[index_original]))
                arr_weights_temp.append(arr_weights[index_original])
       
        pred = 0
        try:
            eclf = VotingClassifier(estimators = estimators_arr_temp, voting='soft') 
            eclf.estimators_ = arr_clf_temp_to_use
            eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))
            pred_proba = eclf.predict_proba(X_train[i, :].reshape(1, -1))
            pred = eclf.predict(X_train[i, :].reshape(1, -1))
            arr_tempo = pred_proba[0]
            arr_pred_win.append(arr_tempo[1])
        except:
            eclf = VotingClassifier(estimators = estimators_arr_temp, voting='hard') 
            eclf.estimators_ = arr_clf_temp_to_use
            eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))
            pred = eclf.predict(X_train[i, :].reshape(1, -1))
            if(pred==0):
                arr_pred_win.append(0)
            if(pred==1):
                arr_pred_win.append(1)  
        
        #print("i:",i)
        ensemble_preds[i] = pred
        #print("ens_size:",len(ensemble_preds))
        #sys.exit()
        counter = i+1
        lambda_0 = maj_weight#Majority
        lambda_1 = min_weight #Minority
        
        if y_train[i] == 0:
            zeroClassCounter += 1
            zeroClassCounter_CWindow += 1
        else: 
            oneClassCounter += 1
            oneClassCounter_CWindow += 1
        correct_found = 0

        for n in range(0,len(arr_clf)):
            if(y_train[i] == 0 and arr_preds[n] ==0):
                current_correct_0_cw[n] = current_correct_0_cw[n]+1
            if(y_train[i] == 1 and arr_preds[n] ==1):    
                current_correct_1_cw[n] = current_correct_1_cw[n]+1
        
        correct_found = 0  
        y_actual = 0
        y_pred = 0
        if(y_train[i] ==0 and pred ==0):
            correct_found +=1
            current_correct_ensemble_0 += 1
            current_correct_0_cw_est += 1
            ensemble_tn += 1
            y_actual = 0
            y_pred = 0
        if(y_train[i] ==1 and pred == 1):
            correct_found +=1
            current_correct_ensemble_1 += 1
            current_correct_1_cw_est += 1
            ensemble_tp += 1
            y_actual = 1
            y_pred = 1
        if(y_train[i] ==1 and pred ==0):
            ensemble_fn += 1
            y_actual = 1
            y_pred = 0
        if(y_train[i] ==0 and pred == 1):
            ensemble_fp += 1
            y_actual = 0
            y_pred = 1
        if(correct_found ==1):
            ensemble_correct += 1
            
        arr_actual_win.append(y_actual)                   
        
        
        #when the batch is ready / an incomplete batch at the end
        if( ( current_w % window_size == 0)  or (current_w != window_size and i == (row_num - w -1)) ):

            window_counter += 1
            
            window_recall_0 = 0
            window_recall_1 = 0
            if zeroClassCounter_CWindow != 0:
                window_recall_0 =  current_correct_0_cw_est / zeroClassCounter_CWindow
            if oneClassCounter_CWindow != 0:
                window_recall_1 =  current_correct_1_cw_est / oneClassCounter_CWindow
            
            window_gmean = sqrt(window_recall_0*window_recall_1)
            
            
            update_recall_weight( (lambda_0*window_recall_0) + (lambda_1*window_recall_1))
            
            #print("current window gmean:",str(window_gmean*100))
            arr_gmean.append(window_gmean*100)
            arr_recall_0.append(window_recall_0*100)
            arr_recall_1.append(window_recall_1*100)
            
            accuracy = (ensemble_tp+ensemble_tn)/(zeroClassCounter_CWindow+oneClassCounter_CWindow)

            arr_actual_win=[]
            arr_pred_win = []
            arr_cw_ens_preds = []            
            
            for n in range(0,len(arr_clf)):                    
                if zeroClassCounter_CWindow != 0:
                    recall_0_cw[n] =  current_correct_0_cw[n] / zeroClassCounter_CWindow                    
                else:
                    recall_0_cw[n] =  1
                    
            for n in range(0,len(arr_clf)):
                    if oneClassCounter_CWindow != 0:
                        recall_1_cw[n] =  current_correct_1_cw[n] / oneClassCounter_CWindow
                    else:
                        recall_1_cw[n] = 1

            arr_weights = [(lambda_0 * recall_0_cw[n]) + (lambda_1 * recall_1_cw[n]) for n in range(len(arr_clf))]

            arr_weights = normalize(arr_weights)

            oneClassCounter_CWindow = 0
            zeroClassCounter_CWindow = 0  
            current_correct_0_cw_est = 0
            current_correct_1_cw_est = 0
            ensemble_tn = 0
            ensemble_tp = 0
            ensemble_fp = 0
            ensemble_fn = 0
            
            current_correct_0_cw = [0] * len(arr_clf)
            current_correct_1_cw = [0] * len(arr_clf)
            
           
            x_preq = X_train[i-(window_size-1):i]
            y_preq = y_train[i-(window_size-1):i]

            Xy_preq = Xy_train[i-(window_size-1):i]
            ups =  np.count_nonzero(y_preq == 1)
            downs = np.count_nonzero(y_preq == 0)            

            arr_clf_temp = []

            ####################### <Overlap Detection> ############################################
            #Check overlaps where and send its status to partialfit
            cols_str = []
            for f in range(1,n_features+1):
                cols_str.append(str(f))
            cols_str.append('class')
            Xy_d = pd.DataFrame(Xy_preq, columns = cols_str)
            if(ups < downs):
                Xy_min = Xy_d[(Xy_d['class'] == 1)]
                Xy_maj = Xy_d[(Xy_d['class'] == 0)]
            if(ups>downs):
                Xy_min = Xy_d[(Xy_d['class'] == 0 )]
                Xy_maj = Xy_d[(Xy_d['class'] == 1 )]

            cols = []
            for f in range(0,n_features):
                cols.append(f)
            cols_class = [n_features]
            X_min = Xy_min[Xy_min.columns[cols]]
            X_maj = Xy_maj[Xy_maj.columns[cols]]
            y_min = Xy_min[Xy_min.columns[cols_class]]
            y_maj = Xy_maj[Xy_maj.columns[cols_class]]
            
            frames = [X_min,X_maj]
            modified_X = pd.concat(frames)
            frames = [y_min,y_maj]
            modified_Y = pd.concat(frames)

            cols_num = []            
            cols_num = list(range(n_features))

            m_X = modified_X
            m_Y = modified_Y            
            m_X.columns = cols_num
            m_Y.columns = [0]  
            
            
            overlap_val, fishers = detect_overlap(X_min, X_maj,n_features)   
            
            ####################### </Drift Detection and Overlap> ############################################
            
            modified_x,modified_y = icrc(X_maj,X_min,y_maj,y_min,n_features,overlap_val,False,ups,downs)
                   
            arr_temp_clfs = []
            ## Partial Fit the previous classifiers
            for n in range(0,len(arr_clf)):
                clf = arr_clf[n]
                clf = clf.partial_fit(modified_x,modified_y)
                arr_temp_clfs.append(clf)
             
            rl_pruned_ids = []
            #################################### Commented out code that removes the minimum preto based member ###############
            arr_clf = arr_temp_clfs
            #remove one classifier if the pool has reached..
            remove_index = 0
            start1 = timer()
            if(len(arr_clf) == ensemble_pool_size):
                least_weight = arr_weights[0]
                for q in range (1,len(arr_weights)):
                    w_q = arr_weights[q]
                    if(w_q < least_weight):
                        remove_index = q
                arr_clf = np.delete(arr_clf,remove_index)
                arr_weights = np.delete(arr_weights,remove_index)
            global elt_global
            end1 = timer()
            elt= end1-start1
            elt_global += elt    
            ######################### add new classifier to the pool after training it with the current window.#####################
            obj_clf1 = DetectorClassifier(HoeffdingTreeClassifier())
            clf_temp =obj_clf1.partial_fit(modified_x,modified_y)
 
            arr_weights = np.append(arr_weights,0)
            arr_clf = np.append(arr_clf,clf_temp)
            
            preq_Qtable = np.delete(preq_Qtable, remove_index, axis=1)
            preq_Qtable = np.delete(preq_Qtable, remove_index, axis=0)
                        
            row_of_ones = np.zeros((1, preq_Qtable.shape[1]), dtype=preq_Qtable.dtype)
            preq_Qtable = np.vstack((preq_Qtable, row_of_ones))

            # Add a column containing ones for all rows
            column_of_ones = np.zeros((preq_Qtable.shape[0], 1), dtype=preq_Qtable.dtype)
            preq_Qtable = np.hstack((preq_Qtable, column_of_ones))

            if(len(arr_clf) == ensemble_pool_size):

                rl_pruned_ids,preq_Qtable = get_pruned_rl_members(arr_clf,x_preq, y_preq,window_counter,preq_Qtable)
            ##########################################################################################################        
            current_w = 0
            prev_win_x = x_preq
            prev_win_y =  y_preq
            
            end = timer()
            elapsed_time = end - start
            arr_elapsedtime.append(elapsed_time)
            print(elapsed_time)
            #if(window_counter % 50 == 0):
            #    np.savetxt(output_folder + file_out_prefix+ "-"  +str(window_counter) +'.csv', np.c_[arr_recall_0,arr_recall_1,arr_gmean], delimiter=',',fmt='%2.5f')
               
    
    np.savetxt(output_folder + file_out_prefix + '.csv',  np.c_[arr_recall_0,arr_recall_1,arr_gmean,arr_elapsedtime], delimiter=',',fmt='%1.2f')
    
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable    
    
def take_step(ensemble,action,X_test,y_test,prev_perf):
    global ensemble_pool_size    
    ensemble.append(action)    
    eclf = VotingClassifier(estimators = ensemble, voting='soft') 
    eclf.estimators_ = ensemble
    eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))

    #print(y_test)
    y_pred = np.zeros(y_test.shape)
    
    if(len(ensemble) == 1):
        eclf = ensemble[0]
        
    y_pred = np.zeros(y_test.shape)
    
    zero_corrects = 0
    zeros_count = 0
    one_corrects = 0
    ones_count = 0

    num_zeros = np.count_nonzero(y_test == 0)
    num_ones = np.count_nonzero(y_test == 1)
       
    if len(X_test.shape) == 1:
        X_test = X_test.reshape(1, -1)  # Reshape to a 2D array with a single row

    # Predict all examples in X_test
    predictions = eclf.predict(X_test)

    # Compare predictions with y_test
    zero_corrects = np.sum((y_test == 0) & (predictions == 0))
    one_corrects = np.sum((y_test == 1) & (predictions == 1))     
   
    recall_0 = zero_corrects / num_zeros
    if(ones_count ==0):
        recall_1 = 0
    else:
        recall_1 = one_corrects / num_ones
    weight_en = recall_0*maj_weight + recall_1*min_weight
    
    reward = weight_en - prev_perf

    new_state = action
    done = False
    if(len(ensemble)>ensemble_pool_size or reward > 0.99):
        done = True
    return new_state, reward, done, 0,weight_en

def read_data(filename):
    df = pd.read_csv(filename)
    data = df.values
    return data, data[:, :-1], data[:, -1]

def delete_files_from_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def train_rl(gamma,learning_rate,min_epsilon, max_epsilon, decay_rate, max_steps, Qtable,X_test,y_test,classifiers,window_counter):
  global rl_episodes 
  global restart_threshold
  global rl_each_step_episodes
  global ensemble_pool_size
  ep = 0 
  rl_episodes_to_use = rl_episodes
  if(window_counter==2):
     rl_episodes_to_use = rl_episodes

  elif( window_counter % restart_threshold ==0):
    rl_episodes_to_use = rl_episodes
  else:
    rl_episodes_to_use = rl_each_step_episodes
  for episode in range(0,rl_episodes_to_use):
    ep += 1
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

    running_ep_val = epsilon
    # Reset the environment
    state_index= random.randint(0, ensemble_pool_size-1)

    state = classifiers[state_index]
    step = 0
    done = False
    ensemble = []
    
    predictive_performance = 0.90
    # repeat
    for step in range(max_steps):
      action_index = epsilon_greedy_policy(Qtable, state_index, epsilon,False)
      if(action_index==state_index or (action_index in ensemble)):
        continue
      ensemble.append(state)

      action = classifiers[action_index]

      new_state, reward, done, info,predictive_performance = take_step(ensemble,action,X_test,y_test,predictive_performance)
      new_state = action_index

      Qtable[state_index][action_index] = Qtable[state_index][action_index] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state_index][action_index])
      # If done, finish the episode
      if done:
        break
     
      # Our state is the new state
      state_index =new_state
      state =  classifiers[new_state]
  return Qtable

def epsilon_greedy_policy(Qtable, state, epsilon,running_ep_val):

    random_int = random.uniform(0,1)
    if running_ep_val:
        action = np.argmax(Qtable[state])
    elif random_int > epsilon:
        action = np.argmax(Qtable[state])
    else:
        action = random.randint(0, ensemble_pool_size-1)

    return action

def get_pruned_rl_members(trained_members,X_test,y_test,window_counter,QTable):

    global restart_threshold
    if(window_counter==2):
        Qtable_ens = initialize_q_table(ensemble_pool_size,ensemble_pool_size)

    elif(window_counter % restart_threshold ==0):
        Qtable_ens = initialize_q_table(ensemble_pool_size,ensemble_pool_size)
    else:
        Qtable_ens = QTable
    
    global learning_rate
    global decay_factor

    # Environment parameters
    max_steps = 15             
    gamma = 0.95               
    eval_seed = []             

    # Exploration parameters
    max_epsilon = 1.0           
    min_epsilon = 0.05           
 
    
    ensemble_arr = []
    weights_arr = []
    Qtable = train_rl(gamma,learning_rate,min_epsilon, max_epsilon, decay_factor, max_steps, Qtable_ens,X_test,y_test,trained_members,window_counter)
    classifiers = trained_members
    rl_pruned_ids = []
    ensemble = []
    state_index= random.randint(0, ensemble_pool_size-1)
    rl_pruned_ids.append(state_index)
    state = classifiers[state_index]
    ensemble.append(state)
    step = 0
    done = False
    ensemble = []

    
    #max_perf
    max_gmean = 0
    max_index = 0
    step_index= -1
    # repeat
    for step in range(max_steps):
        step_index += 1
        action_index = epsilon_greedy_policy(Qtable, state_index,0, True)

        if(action_index==state_index or (action_index in rl_pruned_ids)):
            continue
        
        rl_pruned_ids.append(action_index)

        action = classifiers[action_index]

        ensemble.append(action)

        eclf = VotingClassifier(estimators = ensemble, voting='soft') 
        eclf.estimators_ = ensemble
        eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))

        num_zeros = np.count_nonzero(y_test == 0)
        num_ones = np.count_nonzero(y_test == 1)
            
        if len(X_test.shape) == 1:
            X_test = X_test.reshape(1, -1)  # Reshape to a 2D array with a single row

        # Predict all examples in X_test
        predictions = eclf.predict(X_test)

        # Compare predictions with y_test
        zero_corrects = np.sum((y_test == 0) & (predictions == 0))
        one_corrects = np.sum((y_test == 1) & (predictions == 1))     

        recall_0 = zero_corrects / num_zeros
        if(num_ones ==0):
            recall_1 = 0
        else:
            recall_1 = one_corrects / num_ones
        gmean = math.sqrt (recall_0 * recall_1)
        
        
        
        if(gmean > max_gmean):
            max_index = step_index
            max_gmean = gmean
        
        state_index =action_index
        state =  classifiers[state_index]
        done = False
        if(len(ensemble) == pruned_ensemble_size ):

            break

    return rl_pruned_ids, Qtable              
            
#*********************************  Main ******************************** #

global imbalance_threshold
global arr_recall_0
global arr_recall_1
global arr_gmean
global restart
global restart_threshold
global f1_overlap_value
global maj_weight
global min_weight
global ens_weight
global arr_ord_cols
global preq_Qtable
global restart_threshold
global min_life_max
global min_life_decay_factor
global output_folder
global stream_folder

global ensemble_pool_size
global pruned_ensemble_size
global rl_pruned_ids
global learning_rate
global decay_factor
global minority_buffer_x
global minority_buffer_y
minority_buffer_x = []
minority_buffer_y = []
global df_buffer_x
global rl_each_step_episodes
global arr_elapsedtime
global elt_global
global disable_buffer


elt_global = 0
df_buffer_x = pd.DataFrame()
f1_overlap_value = 0

cwd = os.getcwd()
cwd = "C:/Users/Lenovo/EMRIL"  
#cwd = "D:\\usman-data"
output_folder = cwd +  '/output/'

# 👇 make sure it exists
os.makedirs(output_folder, exist_ok=True)
stream_folder = cwd +  '/data_set/'

arr_elapsedtime = []
arr_gmean = []
arr_recall_0 = []
arr_recall_1 = []
rl_pruned_ids = []

ensemble_pool_size = 15 #ensemble pool size
pruned_ensemble_size =8#pruned ensemble size
restart_threshold = 50 #batches after q-table is emptied
rl_episodes = 5 #on restart
rl_each_step_episodes =1

n_train = 200 #no of instances to train the model (offline)
w =200 #batch size
overlap_threshold_f1 = 0.0 #overlap threshold
imbalance_ratio_threshold =1.01 #IR threshold
disjuncts_threshold =2 #MinD Threshold
disjunct_threshold_perc_maj = 1.0 #overlap removal threshold
maj_weight = 0.1 #lambda_0 majority weight in ensemble
min_weight = 0.9 #Lambda_1 minority weight in ensemble
min_life_max = 0.99 #minoirty instance life max value
min_life_decay_factor = 0.03 #minority instance life decay factor
disable_buffer = False

#/*If data stream contains ordinal cols, provide indices here*/
arr_ord_cols= []
#arr_ord_cols = [6,7,8,9,10] #1-based index  GMSC
#arr_ord_cols = [13,14,15,16,17,18,19,20,21] #1-based index  IJCNN1
#arr_ord_cols = [10,11,12] #1-based index  LOAN
#arr_ord_cols = [1,2] #1-based index  Mixed

#filename = "v5_imb.csv"
#filename = "v9_imb.csv"
#filename = "ijcnn1-full-30oct2022.csv"
#filename = "cod-rna-311022.csv"
#filename = "MiniBooNE_PID_Mod.csv"
#filename =  "GMSC.csv"
#filename = "loan_191122.csv"
filename = "noaa.csv"
#filename = "mixed50K_Static01Abr.csv"


f_arr = filename.split(".")

filename = stream_folder  + filename

Xy, X, y = read_data(filename)


# Set x,y as numeric
X = X.astype(float)
n_samples, n_features = X.shape
print(n_samples)
print(n_features)
ups =  np.count_nonzero(y == 1)
downs = np.count_nonzero(y == 0)
print("1s:",str(ups/n_samples))
print("0s:",str(downs/n_samples))

#create the minority buffer structure
cols_str = []
for f in range(1,n_features+1):
    cols_str.append(str(f))
cols_str.append("life")
cols_str.append("recall_weight")
cols_str.append("total_weight")
df_buffer_x = pd.DataFrame(columns = cols_str)
for k in range (1,11):
    elt_global = 0
    decay_factor = 0.005
    learning_rate = 0.8    
    arr_recall_0= []
    arr_recall_1= []
    arr_gmean= []
    arr_elapsedtime = []
    df_buffer_x = pd.DataFrame(columns = cols_str)
    
    file_out_prefix = "EMRILs_" + f_arr[0] + "_" + str(k) 
    clf = tree.HoeffdingTreeClassifier()

    # Example of usage with stream-based data
    clfs = [clf]
    clfs_label = ["Hoeffding Tree Classifier"]
    for i in range(len(clfs)):
        with np.errstate(divide='ignore', invalid='ignore'):
            prequential(w,Xy,X, y, clfs[i], n_features,overlap_threshold_f1,
                        imbalance_ratio_threshold,disjunct_threshold_perc_maj,file_out_prefix,n_train)
sys.exit()    




18159
8
1s: 0.6862162013326725
0s: 0.0
1.6509418999776244
3.4743025999050587
5.082432699971832
5.924085199949332
6.8449452999047935
7.526902299956419
8.340357499895617
9.165918199927546
9.934717700001784
10.589988399995491
11.238443899899721
12.069579999893904
13.030603799968958
14.107763799955137
15.227222999907099
16.032174699939787
16.738560499972664
17.38073049997911
18.139499899931252
18.871488599921577
19.7406627999153
20.56391749996692
21.38036359997932
22.234897299902514
23.130818799952976
23.898961999919266
24.58865779999178
25.298990199924447
28.211766799911857
30.10611319995951
30.918482299894094
31.747981299995445
32.66764159989543
33.44636169995647
34.11117859999649
34.784039499936625
35.43057419999968
36.064498899970204
36.74866179993842
37.5685903999256
38.31238989997655
39.0270347999176
39.700134399929084
40.39229369990062
41.0237062999513
41.646084699896164
42.31691850000061
42.93497639999259
43.628309799940325
45.05649839993566
46.0322902998887
46.954780299915
47.9896

SystemExit: 